In [ ]:
import pandas as pd
import re
from itertools import chain
from tqdm.auto import tqdm
from transformers import pipeline
from datasets import Dataset  # HuggingFace dataset
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import requests
import nasdaqdatalink
import os
from dotenv import load_dotenv
load_dotenv()
API_KEY= os.getenv('API_KEY')
nasdaqdatalink.ApiConfig.api_key = API_KEY

In [ ]:
df=pd.read_csv('data/preprocessedtranslated_tweets_us24.csv')

In [ ]:
df.head()

,Tweet ID,Username,Text,Date,Retweets,Likes,Weight
0,1798142586911342936,Rabbit1873,JackPosobiec Re BREAKING GARLAND TO LAUNCH ELE...,Tue Jun 04 23:59:35 +0000 2024,2,3,0.622459
1,1798142575112769914,Chrissi Miller,PaulStanleyLive PresidentDonaldTrump wins the ...,Tue Jun 04 23:59:32 +0000 2024,0,0,0.622459
2,1798142544762830918,Pedro Da Conceicao,tomorrowsmps I believe I am the youngest PPC s...,Tue Jun 04 23:59:25 +0000 2024,0,0,0.982014
3,1798142432388714849,Ben 🇺🇲,I am already saying Democrats will cheat even ...,Tue Jun 04 23:58:58 +0000 2024,0,0,0.500000
4,1798142320958943735,🇺🇸Proud American🇮🇹,BryanDeanWright Republican AGs should prosecut...,Tue Jun 04 23:58:31 +0000 2024,0,2,0.880797
...,...,...,...,...,...,...,...
26353,1853587626646978743,Alabaster Slim,DreadOsiris stelzner_n1150 LOL LOL I am a Cons...,Mon Nov 04 23:58:22 +0000 2024,0,0,0.622459
26354,1853587556425777555,Judith L Osborne🌊#Resistance,deadbrainworm JoJoFromJerz No one takes RFK jr...,Mon Nov 04 23:58:06 +0000 2024,0,2,0.500000
26355,1853587519654256996,Robert,baldwin_daniel_ Kamala could not fill a high s...,Mon Nov 04 23:57:57 +0000 2024,0,1,0.817574
26356,1853587506148610556,Elizabeth A 🍁🇨🇦,MaeMaes65 NotOpCue NewsTreason No he did not I...,Mon Nov 04 23:57:54 +0000 2024,0,0,0.880797


In [10]:
tqdm.pandas()
# 2) Initialize a sentiment-analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

Device set to use mps:0


In [11]:
def generate_aliases(person):
    first, last = person["first"], person["last"]
    full = f"{first} {last}"
    aliases = [full, first, last]
    if "nicknames" in person:
        aliases.extend(person["nicknames"])
    if "handle" in person:
        aliases.extend([person["handle"], f"@{person['handle']}"])
    return aliases

# People and slogans
dem_people = [
    {"first": "Joe", "last": "Biden", "nicknames": ["JoeBiden"], "handle": "JoeBiden"},
    {"first": "Kamala", "last": "Harris", "nicknames": ["KamalaHarris"], "handle": "KamalaHarris"},
]
rep_people = [
    {"first": "Donald", "last": "Trump", "nicknames": ["Trump"], "handle": "realDonaldTrump"},
    {"first": "Nikki", "last": "Haley", "nicknames": ["NikkiHaley"], "handle": "NikkiHaley"},
    {"first": "Mike", "last": "Pence", "nicknames": ["Pence"], "handle": "Mike_Pence"},
    {"first": "Vivek", "last": "Ramaswamy", "nicknames": ["Vivek2024"], "handle": "VivekGRamaswamy"},
    {"first": "JD", "last": "Vance", "nicknames": ["JDVance"], "handle": "JDVance1"},
    {"first": "Robert", "last": "Kennedy Jr.", "nicknames": ["RFKJr", "Kennedy2024"], "handle": "RobertKennedyJr"},
]

# Entities
dem_entities = list(chain.from_iterable(generate_aliases(p) for p in dem_people)) + [
    "democrat", "democrats", "Democrats", "democratic party", "dnc", "vote blue", "blue wave", "bidenomics"
]
rep_entities = list(chain.from_iterable(generate_aliases(p) for p in rep_people)) + [
    "republican", "Republican", "republicans", "gop", "rnc", "maga", "trump2024", "drain the swamp"
]

# Regex patterns
def compile_pattern(entities):
    sorted_ents = sorted(set(entities), key=len, reverse=True)
    pat = r'\b(' + '|'.join(re.escape(ent) for ent in sorted_ents) + r')\b'
    return re.compile(pat, flags=re.IGNORECASE)

dem_pattern = compile_pattern(dem_entities)
rep_pattern = compile_pattern(rep_entities)

In [12]:
def truncate_sent(text, max_len=512):
    if not isinstance(text, str):
        return ""  # or return a placeholder like "[no text]"
    return text if len(text) <= max_len else text[:max_len]

In [13]:
ds = Dataset.from_pandas(df[['Text']])  # Just keep the needed column

# Load sentiment model (batch capable)
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", device=0)  # if GPU available

# Apply sentiment in batch
# 2. If you worry about super‑long texts, truncate in Python:
# 2) Apply to all tweets
truncated_texts = [truncate_sent(t) for t in ds['Text']]

# 3) Run sentiment analysis in batch
sentiments = sentiment_analyzer(truncated_texts, batch_size=32)

ds = ds.add_column("Sentiment", sentiments)

Device set to use mps:0


In [ ]:
df_s = ds.to_pandas()
df['Date'] = pd.to_datetime(df['Date'])

In [24]:
# 4) Convert to DataFrame and match date including exact minute/second current format: Tue Jun 04 23:59:32 +0000 2024

df['Score'] = df_s['Sentiment'].apply(lambda x: x['score'])
df['Label'] = df_s['Sentiment'].apply(lambda x: x['label'])
df

,Tweet ID,Username,Text,Date,Retweets,Likes,Weight,Score,Label
0,1798142586911342936,Rabbit1873,JackPosobiec Re BREAKING GARLAND TO LAUNCH ELE...,2024-06-04 23:59:35+00:00,2,3,0.622459,0.936691,NEGATIVE
1,1798142575112769914,Chrissi Miller,PaulStanleyLive PresidentDonaldTrump wins the ...,2024-06-04 23:59:32+00:00,0,0,0.622459,0.998360,POSITIVE
2,1798142544762830918,Pedro Da Conceicao,tomorrowsmps I believe I am the youngest PPC s...,2024-06-04 23:59:25+00:00,0,0,0.982014,0.830334,POSITIVE
3,1798142432388714849,Ben 🇺🇲,I am already saying Democrats will cheat even ...,2024-06-04 23:58:58+00:00,0,0,0.500000,0.569910,NEGATIVE
4,1798142320958943735,🇺🇸Proud American🇮🇹,BryanDeanWright Republican AGs should prosecut...,2024-06-04 23:58:31+00:00,0,2,0.880797,0.995128,NEGATIVE
...,...,...,...,...,...,...,...,...,...
26353,1853587626646978743,Alabaster Slim,DreadOsiris stelzner_n1150 LOL LOL I am a Cons...,2024-11-04 23:58:22+00:00,0,0,0.622459,0.997468,NEGATIVE
26354,1853587556425777555,Judith L Osborne🌊#Resistance,deadbrainworm JoJoFromJerz No one takes RFK jr...,2024-11-04 23:58:06+00:00,0,2,0.500000,0.998815,NEGATIVE
26355,1853587519654256996,Robert,baldwin_daniel_ Kamala could not fill a high s...,2024-11-04 23:57:57+00:00,0,1,0.817574,0.999561,NEGATIVE
26356,1853587506148610556,Elizabeth A 🍁🇨🇦,MaeMaes65 NotOpCue NewsTreason No he did not I...,2024-11-04 23:57:54+00:00,0,0,0.880797,0.972673,NEGATIVE


In [26]:
relevant_tweets = df[df['Likes'] > 100]
relevant_tweets

,Tweet ID,Username,Text,Date,Retweets,Likes,Weight,Score,Label
13,1798504961107783798,Collin Rugg,NEW Rapper 50 Cent says he sees young African ...,2024-06-05 23:59:31+00:00,884,7733,0.880797,0.683100,NEGATIVE
14,1798504952744345841,.*Funkytown™*.,50 Cent says black men identify with Trump bec...,2024-06-05 23:59:29+00:00,164,316,0.500000,0.982600,NEGATIVE
60,1799954447843426447,🇺🇸 Larry 🇺🇸,NEW watch as a CNN host asks the tough questio...,2024-06-09 23:59:16+00:00,96,455,0.880797,0.781170,POSITIVE
103,1800677354928484553,Charlie Bilello,The SP 500 is up 127 in the first 112 trading ...,2024-06-11 23:51:51+00:00,171,659,0.880797,0.999680,POSITIVE
193,1802851766477963610,Chuck Callesto,Democrats RINOs The Deep State The 2020 elec...,2024-06-17 23:52:11+00:00,91,329,0.500000,0.986592,NEGATIVE
...,...,...,...,...,...,...,...,...,...
25993,1845613574120497290,Footballmema,Daudlp02 RpsAgainstTrump God bless America God...,2024-10-13 23:52:20+00:00,42,304,0.880797,0.667019,POSITIVE
26024,1846339337413083266,End Tribalism in Politics,Mark Halperin I think it is a mistake to just ...,2024-10-15 23:56:16+00:00,324,2408,0.952574,0.999579,NEGATIVE
26038,1846338021978693937,jude,Imagine going back to 2004 and telling someone...,2024-10-15 23:51:02+00:00,20,117,0.880797,0.968758,NEGATIVE
26267,1851775646768468285,Homer Hickam,robert_zubrin You are rarely wrongRobert but t...,2024-10-30 23:58:13+00:00,4,189,0.880797,0.968108,POSITIVE
